Import the libraries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gradio as gr
import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Dense, Embedding, Input,  Activation
from keras.layers import  GlobalMaxPool1D, Dropout
from keras import  optimizers, layers
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.metrics import Precision, Recall, CategoricalAccuracy
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D

In [2]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = stopwords.words('english')
import html
import unicodedata
import html
from tqdm.auto import tqdm
import re
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\toof-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\toof-\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\toof-\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\toof-\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


1: Uploading the dataset

In [3]:
# upload the data from file 
train_df = pd.read_csv('train.csv')

In [4]:
# Generating 10 samples from the file
train_df.sample(10, random_state = 1234)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
12345,20bc45f5014c1eca,"""\n\nYour latest edit has again had to be remo...",0,0,0,0,0,0
55587,948580166b8b8745,"""\n\nWorks for me. BaStun not BaTsun """,0,0,0,0,0,0
16877,2c8519885db5c0bb,List of newspapers in India by readership\n\nT...,0,0,0,0,0,0
140039,ed64d530a34aa973,Happy New Years\nCheers ),0,0,0,0,0,0
125777,a0dd897eafbada2c,Photos \nSouthwestern’s official website has m...,0,0,0,0,0,0
159512,ff2f5695bb9aa845,"How's this? It's a lot shorter, but it's more ...",0,0,0,0,0,0
121612,8aa46e67645e4b7b,Why did you remove skinwalker instead of movin...,0,0,0,0,0,0
7588,142d9e745d03c9ef,"""I repeat, the text was about a minority schoo...",0,0,0,0,0,0
9463,191a1c6bd40ad470,bay pigs article \n\nhi i noticed that you wer...,1,0,1,0,0,0
103236,287b2d375ed7188a,"MusikAnimal is a liar, a disinformant and plag...",0,0,0,0,0,0


In [5]:
# if cond lable == toxic bering it 
train_df.loc[train_df['toxic'] == 1].sample(10, random_state=1234)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
86296,e6cfe09510b15145,and are Angelique KKKarrington-hating Jews!]],1,0,0,0,0,0
8619,16e110672dde00d9,How civil is this - who the fuck do you think ...,1,1,1,0,1,0
123695,958e4d243a743156,"""\n\n idiot \n\n i do not understand what use...",1,0,1,0,1,0
120058,820861d281284864,"""::I'd hoped for a better, more neutral respon...",1,0,0,0,0,0
157801,e381fcf1759b1d0f,"""\n\nComments from the west are irrelevant, we...",1,0,0,0,0,0
132607,c57242ec5816d21c,"""\n\nYOUR MOM IS SO FAT, SHE GETS CASHBACK WHE...",1,1,1,0,1,1
73811,c580d59f2a359dcc,"[User talk:JesseRafe|talk]], How dare you call...",1,0,0,0,1,0
17995,2f85b9f5219235df,"to be honest with you, if you don't tell me a ...",1,0,0,0,0,0
45799,7a675e86c2e62890,"P.S. I need some penis, cat tongue is getting old",1,0,1,0,0,0
148162,47cf25a86ba2dc49,FUCK ALL MODERNISTS \n\nI WISH THE ROMAN EMPIR...,1,0,1,0,0,0


2: Cleaning the comments

In [7]:
def remove_special_chars(text):
    re1 = re.compile(r'  +')
    x1 = text.lower().replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>', 'u_n').replace(' @.@ ', '.').replace(
        ' @-@ ', '-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x1))


def to_lowercase(text):
    return text.lower()



def remove_punctuation(text):
    """Remove punctuation from list of tokenized words"""
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

### search for it 
def replace_numbers(text):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    return re.sub(r'\d+', '', text)

def remove_whitespaces(text):
    return text.strip()


def remove_stopwords(words, stop_words):
    """
    :param words:
    :type words:
    :param stop_words: from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
    or
    from spacy.lang.en.stop_words import STOP_WORDS
    :type stop_words:
    :return:
    :rtype:
    """
    return [word for word in words if word not in stop_words]

#####
#def stem_words(words):
   # """Stem words in text"""
    #stemmer = PorterStemmer()
    #return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    """Lemmatize words in text"""

    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(word) for word in words]

def lemmatize_verbs(words):
    """Lemmatize verbs in text"""

    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word, pos='v') for word in words])

def text2words(text):
    return word_tokenize(text)

def cleaning_text( text):
    text = remove_special_chars(text)
    text = remove_punctuation(text)
    text = to_lowercase(text)
    text = replace_numbers(text)
    words = text2words(text)
    words = remove_stopwords(words, stop_words)
    #words = stem_words(words)# Either stem ovocar lemmatize
    words = lemmatize_words(words)
    words = lemmatize_verbs(words)

    return ''.join(words)

In [8]:
# apply cleaning in all comment_text in dataset مرور  (understand  lambda )
train_df['comment_text'] = train_df['comment_text'].apply(lambda x: cleaning_text(x))

3: preprocessing

In [9]:
# cleaning comment in x 
X = train_df['comment_text'] 
# y conten the class for a sntences == lables
y = train_df[train_df.columns[2:]].values

In [10]:
# initialize a TextVectorization layer
# output_sequence_length every sentance we take from the dataset we convert to 600 length
# output_mode to convert each single word to integer
vocab=20000
vectorizer = TextVectorization(max_tokens=vocab,
                               output_sequence_length=600,
                               output_mode='int')

In [11]:
# apabt and teach vectorizer our vocabulary
vectorizer.adapt(X.values)

In [12]:
# vectorizer the words , convert the words to int then save it in vectorized_text
vectorized_text = vectorizer(X.values)

4: Create the model

In [13]:
#MCSHBAP -basic data pipeline generation set # Tow ways to instantiate the data pipline (from_tensor slices) or (list_file)
# pass through our data that just we created ( vectorized_text is the input features ) & ( y is the target lable)
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))# creating the dataset
dataset = dataset.cache()# subsequent data pre_processing
dataset = dataset.shuffle(160000)# how large our buffer needs to be 
dataset = dataset.batch(16)# have each batch represented  as a series of 16 samples 
dataset = dataset.prefetch(8) # helps bottlenecks
#to build data pipline

In [14]:
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [15]:
# Define parameters
filters = 150
kernel_size = 3
hidden_dims = 150


In [16]:
#Create the Embedding Layer
model = Sequential()
# Create the embedding layer 
model.add(Embedding(vocab+1, 32))
#model.add(Bidirectional(LSTM(32, activation='tanh')))
model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, activation = 'relu'))
model.add(MaxPooling1D())
model.add(Conv1D(filters, kernel_size, activation = 'relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims, activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(6, activation = 'sigmoid'))

# Display the model structure
model.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          640032    
                                                                 
 dropout (Dropout)           (None, None, 32)          0         
                                                                 
 conv1d (Conv1D)             (None, None, 150)         14550     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 150)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, None, 150)         67650     
                                                                 
 global_max_pooling1d (Globa  (None, 150)              0         
 lMaxPooling1D)                                         

5: Training the model

In [17]:
# loss = error function , optimize the model  
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# train the model , epochs the num of traning and its random value 
batch_size = 64
history = model.fit(train , epochs=70 , batch_size = batch_size , validation_data=val)

6: Preducation & Evaulation 

In [19]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()

In [20]:
for batch in test.as_numpy_iterator(): 
    # Unpack the batch 
    # X_true = comment in text 
    # y_true = label for these comment 
    # batch = display comment and lebal
    X_true,y_true = batch
    
    # Make a prediction 
    # inter the x_true = comment and see the preduct come with val
    yhat = model.predict(X_true)
    
    # read about it ***
    # Flatten the predictions
    y_true = y_true.flatten()
    yhat = yhat.flatten()
    
    # compare betwenn the result y_true which is the true one  and yhat which the predect one from the model 
    pre.update_state(y_true, yhat)
    re.update_state(y_true, yhat)
    acc.update_state(y_true, yhat)

1/1 [==============================] - 0s 26ms/step


In [21]:
print(f'Precision: {pre.result().numpy()}, Recall:{re.result().numpy()}, Accuracy:{acc.result().numpy()}')

Precision: 0.938470721244812, Recall:0.8987414240837097, Accuracy:0.5315948128700256


Save the model

In [ ]:
model.save('toxicityNLP_Project10.h5')

Loading the model

In [18]:
model = tf.keras.models.load_model('toxicityNLPModel#9.h5')

7: Testing

In [22]:
def score_comment(comment):
    vectorized_comment = vectorizer(comment)
    results = model.predict(np.expand_dims(vectorized_comment,0))
    
    text = ''
    for idx, col in enumerate(train_df.columns[2:]):
        if results[0][idx]>0.5:  #the result of our predect if > 0.5 is toxic 
           text += '{}\n'.format(col)
    if text == '':
      return 'Your comment dose not have any toxic'
    else: 
      return 'Your comment is : \n '+text + '( You should choose your words carefully,please be kind ! )'

In [27]:
interface = gr.Interface(fn=score_comment, 
                         inputs=gr.inputs.Textbox(lines=2, placeholder='Comment to score'),
                         outputs='text')

c:\Users\toof-\anaconda3\lib\site-packages\gradio\inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\toof-\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
c:\Users\toof-\anaconda3\lib\site-packages\gradio\deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)


In [28]:
# just to run our interfce  
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://63eca069-3e8c-4805.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 36ms/step
